<a href="https://colab.research.google.com/github/09aryan/MEME_CLASSIFICATION_USING_SENTIMENT_ANALYSIS/blob/main/InceptionV3model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import csv
import time
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from google.colab import drive

def process_image(img_path, model, target_size=(299, 299)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = tf.image.resize(img_array, (299, 299))
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    features = model.predict(img_array)

    if tf.rank(features) == 4:
        features = tf.reduce_mean(features, axis=(1, 2))
    else:
        # Handle the case where the features have only 2 dimensions
        features = tf.reduce_mean(features, axis=1)

    return features.numpy().tolist()

def process_images_in_colab(model, folder_path, target_size=(299, 299), batch_size=32):
    image_data = []
    img_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path)
                 if filename.endswith((".jpg", ".jpeg", ".png"))]

    start_time = time.time()

    for i, img_path in enumerate(img_paths):
        try:
            features = process_image(img_path, model, target_size=target_size)

            image_data.append({
                'ImageName': os.path.basename(img_path),
                'Features': features
            })

            if (i + 1) % batch_size == 0:
                elapsed_time = time.time() - start_time
                print(f"Processed {i + 1} images in {elapsed_time:.2f} seconds")

        except Exception as e:
            print(f"Error processing image {img_path}: {str(e)}")

    return image_data

def save_to_csv(image_data, csv_file_name):
    with open(csv_file_name, mode='w', newline='') as csvfile:
        fieldnames = ['ImageName', 'Features']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for data in image_data:
            writer.writerow(data)

drive.mount('/content/drive')

base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
output = Dense(2048, activation='linear')(x)

model = Model(inputs=base_model.input, outputs=output)

folder_path = '/content/drive/MyDrive/images'
image_data = process_images_in_colab(model, folder_path, target_size=(299, 299), batch_size=32)
csv_file_name = '/content/drive/MyDrive/image_data_fine_tuned.csv'
save_to_csv(image_data, csv_file_name)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 209ms/step
Processed 2144 images in 869.09 seconds
1/1 [==============================] - 0s 215ms/step
Error processing image /content/drive/MyDrive/images/image_5119.png: image file is truncated
1/1 [==============================] - 0s 375ms/step
Processed 2176 images in 880.97 seconds
1/1 [==============================] - 0s 209ms/step
Processed 2208 images in 893.24 seconds
1/1 [==============================] - 0s 207ms/step
Processed 2240 images in 906.68 seconds
1/1 [==============================] - 0s 210ms/step
Processed 2272 images in 920.16 seconds
1/1 [==============================] - 0s 220ms/step
Processed 2304 images in 933.14 seconds
1/1 [==============================] - 0s 371ms/step
Processed 2336 images in 946.99 seconds
1/1 [==============================] - 0s 345ms/step
Processed 2368 images in 958.68 seconds
1/1 [==============================] - 0s 219ms/step
Proces

In [ ]:
import pandas as pd

# Load the CSV files into pandas DataFrames
labels_df = pd.read_csv('/content/drive/MyDrive/labels.csv')
image_data_df = pd.read_csv('/content/drive/MyDrive/image_data_fine_tuned.csv')

# Merge the DataFrames based on the 'image_name' and 'ImageName' columns
combined_df = pd.merge(labels_df, image_data_df, left_on='image_name', right_on='ImageName', how='inner')

# Select only the columns of interest
result_df = combined_df[['image_name', 'text_ocr', 'text_corrected', 'overall_sentiment', 'Features']]

# Save the result to a new CSV file
result_df.to_csv('/content/drive/MyDrive/combined_data.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Load the combined dataset
df = pd.read_csv('/content/drive/MyDrive/combined_data.csv')

# Preprocess the sentiment labels for multi-class classification
le = LabelEncoder()
df['overall_sentiment'] = le.fit_transform(df['overall_sentiment'])

# Split the dataset into features (X) and labels (y)
X = np.vstack(df['Features'].apply(eval).values)
y = df['overall_sentiment'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a sentiment analysis model for multi-class classification
num_classes = len(le.classes_)
model = Sequential([
    Dense(512, activation='relu', input_dim=X_train_scaled.shape[1]),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Convert labels to one-hot encoding for multi-class classification
y_train_one_hot = to_categorical(y_train, num_classes)
y_test_one_hot = to_categorical(y_test, num_classes)

# Train the model
model.fit(X_train_scaled, y_train_one_hot, epochs=50, batch_size=100, validation_split=0.2)

# Evaluate the model
accuracy = model.evaluate(X_test_scaled, y_test_one_hot)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/50
45/45 [==============================] - 2s 14ms/step - loss: 1.6136 - accuracy: 0.2146 - val_loss: 1.6003 - val_accuracy: 0.4350
Epoch 2/50
45/45 [==============================] - 0s 9ms/step - loss: 1.6008 - accuracy: 0.3185 - val_loss: 1.5871 - val_accuracy: 0.4350
Epoch 3/50
45/45 [==============================] - 0s 9ms/step - loss: 1.5880 - accuracy: 0.3925 - val_loss: 1.5751 - val_accuracy: 0.4350
Epoch 4/50
45/45 [==============================] - 0s 9ms/step - loss: 1.5757 - accuracy: 0.4272 - val_loss: 1.5642 - val_accuracy: 0.4350
Epoch 5/50
45/45 [==============================] - 0s 8ms/step - loss: 1.5629 - accuracy: 0.4364 - val_loss: 1.5529 - val_accuracy: 0.4350
Epoch 6/50
45/45 [==============================] - 0s 9ms/step - loss: 1.5507 - accuracy: 0.4382 - val_loss: 1.5411 - val_accuracy: 0.4350
Epoch 7/50
45/45 [==============================] - 0s 8ms/step - loss: 1.5379 - accuracy: 0.4440 - val_loss: 1.5287 - val_accuracy: 0.4350
Epoch 8/50
45/45 [=

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

# Load the combined dataset
df = pd.read_csv('/content/drive/MyDrive/combined_data.csv')

# Preprocess the sentiment labels for multi-class classification
le = LabelEncoder()
df['overall_sentiment'] = le.fit_transform(df['overall_sentiment'])

# Split the dataset into features (X) and labels (y)
X = np.vstack(df['Features'].apply(eval).values)
y = df['overall_sentiment'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Learning rate scheduler
def lr_schedule(epoch):
    return 0.00001 * 0.9**epoch

lr_scheduler = LearningRateScheduler(lr_schedule)

# Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Build a sentiment analysis model for multi-class classification
num_classes = len(le.classes_)
model = Sequential([
    Dense(512, activation='relu', input_dim=X_train_scaled.shape[1]),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

# Compile the model with RMSprop optimizer and categorical crossentropy loss
model.compile(optimizer=RMSprop(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with learning rate scheduler and early stopping
model.fit(X_train_scaled, y_train, epochs=100, batch_size=100, validation_split=0.2, callbacks=[lr_scheduler, early_stopping])

# Evaluate the model
accuracy = model.evaluate(X_test_scaled, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/100
45/45 [==============================] - 3s 27ms/step - loss: 2.2637 - accuracy: 0.2088 - val_loss: 1.6054 - val_accuracy: 0.2735 - lr: 1.0000e-05
Epoch 2/100
45/45 [==============================] - 0s 10ms/step - loss: 2.3064 - accuracy: 0.1944 - val_loss: 1.5981 - val_accuracy: 0.3130 - lr: 9.0000e-06
Epoch 3/100
45/45 [==============================] - 0s 10ms/step - loss: 2.2701 - accuracy: 0.2034 - val_loss: 1.5981 - val_accuracy: 0.3022 - lr: 8.1000e-06
Epoch 4/100
45/45 [==============================] - 1s 14ms/step - loss: 2.2289 - accuracy: 0.2119 - val_loss: 1.6058 - val_accuracy: 0.2771 - lr: 7.2900e-06
Epoch 5/100
45/45 [==============================] - 1s 16ms/step - loss: 2.2518 - accuracy: 0.2079 - val_loss: 1.6142 - val_accuracy: 0.2502 - lr: 6.5610e-06
Epoch 6/100
45/45 [==============================] - 1s 15ms/step - loss: 2.2333 - accuracy: 0.2054 - val_loss: 1.6227 - val_accuracy: 0.2430 - lr: 5.9049e-06
Epoch 7/100
45/45 [===========================

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from nltk.corpus import stopwords

# Load your dataset
# Assuming df is your DataFrame with columns: 'Features' and 'overall_sentiment'
df = pd.read_csv('/content/drive/MyDrive/combined_data.csv')

# Convert 'overall_sentiment' to one-hot encoding
df_one_hot = pd.get_dummies(df['overall_sentiment'], prefix='sentiment')

# Concatenate the one-hot encoding columns with the original DataFrame
df = pd.concat([df, df_one_hot], axis=1)

# Extract features and labels
X = df['Features']
y = df['overall_sentiment']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Text cleaning and preprocessing
stop_words = set(stopwords.words('english'))
X_train = X_train.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
X_test = X_test.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Convert text data to numerical features using TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Use SMOTE to address class imbalance on the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

# Initialize a Support Vector Machine (SVM) classifier and perform grid search
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
svm_classifier = SVC()
grid_search = GridSearchCV(svm_classifier, param_grid, cv=5)
grid_search.fit(X_train_smote, y_train_smote)

# Get the best parameters
best_params = grid_search.best_params_
print(f'Best Parameters: {best_params}')

# Train the SVM classifier with the best parameters
svm_classifier = SVC(**best_params)
svm_classifier.fit(X_train_smote, y_train_smote)

# Make predictions on the test set
svm_predictions = svm_classifier.predict(X_test_tfidf)

# Evaluate the SVM model
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_classification_rep = classification_report(y_test, svm_predictions)

print('SVM Model:')
print(f'Accuracy: {svm_accuracy}')
print('Classification Report:')
print(svm_classification_rep)

# Try a Random Forest classifier as well
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_smote, y_train_smote)
rf_predictions = rf_classifier.predict(X_test_tfidf)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_classification_rep = classification_report(y_test, rf_predictions)

print('Random Forest Model:')
print(f'Accuracy: {rf_accuracy}')
print('Classification Report:')
print(rf_classification_rep)


Best Parameters: {'C': 10, 'kernel': 'rbf'}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM Model:
Accuracy: 0.07890961262553801
Classification Report:
               precision    recall  f1-score   support

     negative       0.07      0.98      0.14       103
      neutral       0.00      0.00      0.00       405
     positive       0.41      0.01      0.03       657
very_negative       0.00      0.00      0.00        23
very_positive       0.00      0.00      0.00       206

     accuracy                           0.08      1394
    macro avg       0.10      0.20      0.03      1394
 weighted avg       0.20      0.08      0.02      1394

Random Forest Model:
Accuracy: 0.02295552367288379
Classification Report:
               precision    recall  f1-score   support

     negative       0.00      0.00      0.00       103
      neutral       0.00      0.00      0.00       405
     positive       0.41      0.01      0.03       657
very_negative       0.02      1.00      0.03        23
very_positive       0.00      0.00      0.00       206

     accuracy                   

In [ ]:
import nltk
nltk.download('stopwords', download_dir='/usr/share/nltk_data')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True